##  **Voting methods**

In this practical, you will experiment several voting methods on real world data, namely 


* either on the 2002 Irish General Election (https://en.wikipedia.org/wiki/Dublin_North_(D%C3%A1il_constituency)#2002_general_election), on Dublin North data
* or on the 2017 French presidentail election, on the data provided by the city of Crolles and on the data provided in martinique (ecole dillon)  (see https://vote.imag.fr/about et https://zenodo.org/record/1199545/files/voter-autrement-online.pdf?download=1   and https://hal.univ-antilles.fr/hal-01965660/document pour plus de details)for more details)

(you can choose on work either on the Irish general election, or on the french presidential election or on both)

All the data files are on moodle

**French Election, 11 candidates :  file SVT111.cvs** contains the preference order of the voters -  each voter gives 11 to his/her preferred candidate, 10 to the second, etc. The candidate(s) not approved by the voter receive rate 0. Each line corresponds toa voter

**Irish Election, 12 candidates :  file Irish.cvs** contain the preference order of the voters -  each voter gives 12 to his/her preferred candidate, 11 to the second, etc.The candidate(s) not approved by the voter receive rate 0.  Notice that the first column of a line indicate the number of voters whixh expressed the ballot described by the line

For questions 1, 2 and 3 you can use an excel/calc sheet  




**Question 1: determine the winner according to Borda's rule**



*Add your result to the google sheet*  https://docs.google.com/spreadsheets/d/1ZkSVoET8AtS-BCUYmYPo1VQAQsTCOf0Xjtz_Xxd5-L8/edit?usp=sharing

**Question 2: : determine the winner according to Transferable voting  (IRV)**

Add your result to the google sheet https://docs.google.com/spreadsheets/d/1ZkSVoET8AtS-BCUYmYPo1VQAQsTCOf0Xjtz_Xxd5-L8/edit?usp=sharing





**Question 3: let us now explore condorcet coherent methods**

Summary data are provided by files Dillon.cvs   and IrishCondorcet.cvs


Determine whether there is a Condorcet winner and compute the order on the candidates given by Copeland's rule



*Add your result to the google sheet*  https://docs.google.com/spreadsheets/d/1ZkSVoET8AtS-BCUYmYPo1VQAQsTCOf0Xjtz_Xxd5-L8/edit?usp=sharing

##  **e-voting**  (dead line : 30th of november)

Implement a python toolbox which allows the computation, for any file  the format irish.cvs/SVT111.cvs of

    
    * The Condorcet winner (if any)
    * Copeland's preference order
    * Kramer simpson's preference order
    * Borda  preference order
    * STV  preference order
    * The winner elected by the plurality rule
    
    

Test your code on irish.cvs, SVT111.cvs  and SVT411.cvs 

**Your toolbox + demo will be provided under the form of a notebook, for november the 30th - on moodle**

    

In [55]:
## Bordas Rule

import pandas as pd
import numpy as np

df = pd.read_csv('Irish.csv', skiprows=range(4),encoding='latin1')

dfnew = df.copy()

for col in df.columns[1:]:
    dfnew[col] = df['number of voters']*df[col]
        
dfnew
df['number of voters'].sum()

candidates = df.columns[1:]
sumI = [dfnew[can].sum() for can in candidates]

sumI = np.array(sumI)

orderid = np.argsort(sumI)[::-1]

np.sort(sumI)[::-1]

[(candidates[i],sumI[i]) for i in orderid]


[('Trevor Sargent G.P. ', 261376.0),
 ('Sean Ryan Lab ', 223554.0),
 ('Jim Glennon F.F. ', 199348.0),
 ('Michael Kennedy F.F. ', 196552.0),
 ('G.V. Wright F.F. ', 186029.0),
 ('Clare Daly S.P. ', 171961.0),
 ('Nora Owen F.G. ', 146013.0),
 ('Cathal Boland F.G. ', 87491.0),
 ('Ciaran Goulding Non-P ', 61744.0),
 ('Mick Davis S.F. ', 56420.0),
 ('Eamonn Quinn Non-P ', 24170.0),
 ('David Henry Walshe C.C. Csp ', 13841.0)]

In [33]:
## The Condorcet winner (if any)

import pandas as pd
import numpy as np

def append_value(dict_obj, key, value):
    if key in dict_obj:
        if not isinstance(dict_obj[key], list):
            dict_obj[key] = [dict_obj[key]]
        dict_obj[key].append(value)
    else:
        dict_obj[key] = [value]


df = pd.read_csv('Irish.csv', skiprows=range(4),encoding='latin1')
dfResult = df.copy()
df = df.drop("number of voters",1)

finalData = {}

for i in df.columns:
    for j in dfResult.columns[1:]:
        if i != j:
            totalI = 0
            totalJ = 0
            totalI = sum(np.where(dfResult[i] >= dfResult[j], dfResult['number of voters'], 0))
            totalJ = sum(np.where(dfResult[i] <= dfResult[j], dfResult['number of voters'], 0))
            if totalI > totalJ:
                append_value(finalData, i, totalI)
            elif totalI < totalJ:
                append_value(finalData, j, totalJ) 
            else:
                append_value(finalData, i, totalI)
                append_value(finalData, j, totalJ)                
    dfResult = dfResult.drop(i, 1)


sorted_data = sorted(finalData, key=lambda k: len(finalData[k]), reverse=True)

print(sorted_data[0] + "is the Condorcet winner")



Trevor Sargent G.P. is the Condorcet winner


In [53]:
## KS rule

import pandas as pd
import numpy as np

df = pd.read_csv('Irish.csv', skiprows=range(4),encoding='latin1')
columns = df.columns[1:]

finalData = {}

for i in range(len(columns)):
    finalScore = 0
    for j in range(len(columns)):
        if i != j:
            totalJ = 0
            # compare the ranks of each pairs and sum up the voters 
            totalJ = sum(np.where(df[columns[j]] >= df[columns[i]], df["number of voters"], 0))
            # store the highest voters             
            if( finalScore < totalJ):
                finalScore = totalJ
        # decide the KS(winner) for each candidate 
        if(j == len(columns) -1) :
            key = "KS("+columns[i]+")"
            finalData[key] = finalScore

sort_orders = sorted(finalData.items(), key=lambda x: x[1])

print("The order of the candidates elected in Kramer Simpson's rule are:")

for i in sort_orders:
    print(i[0] + " - ", i[1])

The order of the candidates elected in Kramer Simpson's rule are:
KS(Trevor Sargent G.P. ) -  26641.0
KS(Sean Ryan Lab ) -  29198.0
KS(Michael Kennedy F.F. ) -  31830.0
KS(Jim Glennon F.F. ) -  31933.0
KS(Clare Daly S.P. ) -  32716.0
KS(G.V. Wright F.F. ) -  33097.0
KS(Nora Owen F.G. ) -  34965.0
KS(Cathal Boland F.G. ) -  39576.0
KS(Mick Davis S.F. ) -  40403.0
KS(Ciaran Goulding Non-P ) -  40548.0
KS(Eamonn Quinn Non-P ) -  42519.0
KS(David Henry Walshe C.C. Csp ) -  43148.0
